In [1]:
from lxml import etree

In [2]:
parser = etree.HTMLParser()
root = './pages/pag'

In [3]:
from unicodedata import normalize

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [4]:
# run into the element recursively getting the text
def get_element_text(elem):
    if len(elem) == 0:
        return ("\n" if elem.tag == "br" else elem.text)
    if len(elem) == 1:
        if elem[0].tail == None or elem[0].tail.strip() == "":
            return get_element_text(elem[0])
        else:
            if elem[0].text == None:
                return ("\n" if elem[0].tag == "br" else "") + elem[0].tail.strip()
            else:
                return elem[0].text + elem[0].tail.strip()
    else:
        if elem[0].tag == 'br':
            return "\n" + get_element_text(elem[1:])
        if elem[0].tail != None and elem[0].tail.strip() != "":
            return ("" if elem[0].text == None else elem[0].text) + ("" if len(elem[0]) == 0 else get_element_text(elem[0][:])) + elem[0].tail + get_element_text(elem[1:])
        else:
            return ("" if elem[0].text == None else elem[0].text) + ("" if len(elem[0]) == 0 else get_element_text(elem[0][:])) + get_element_text(elem[1:])

### El País

In [5]:
xpath_movie_detail1='//*[@id="sumario_1|despiece"]/div/div/p'
xpath_movie_detail2='//*[@id="sumario_2|despiece"]/div/div/p'
xpath_movie_title='//*[@id="sumario_1|despiece"]/div/header/h4'

filmes =[]
for i in range (1,11):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    r=tree.xpath(xpath_movie_title)
    if r != []:
        filme['nome'] = get_element_text(r)
    r=tree.xpath(xpath_movie_detail2)
    if len(r) == 0:
        r=tree.xpath(xpath_movie_detail1)

    if len(r) > 0:
        for x, item in enumerate(r):
            texto = get_element_text(item)
            if x == 0 and "nome" not in filme:
                filme["nome"] = texto
                continue
            
            try:
                indice = texto.index(":")
            except ValueError:
                indice = -1
            
            if(indice > -1):
                categoria = texto[:indice].lower()
                conteudo = texto[indice+1:].strip()
                
                if categoria == "direção":
                    filme["direcao"] = conteudo
                elif categoria == "gênero":
                    filme["genero"] = conteudo
                elif categoria == "elenco" or categoria == "intérpretes":
                    filme["elenco"] = conteudo
                elif categoria == "duração":
                    filme["duracao"] = conteudo
                elif "outro" in filme:
                    filme["outro"] += texto +";"
                else:
                    filme["outro"] = texto +";"
            elif "outro" in filme:
                filme["outro"] += texto +";"
            else:
                filme["outro"] = texto +";"
        
        filmes.append(filme)
filmes

[{'nome': 'HOMEM-FORMIGA E A VESPA',
  'direcao': 'Peyton Reed.',
  'elenco': 'Paul Rudd, Evangeline Lilly, Michael Douglas, Michelle Pfeiffer.',
  'genero': 'ação. EUA, 2018.',
  'duracao': '118 minutos.'},
 {'nome': 'NICO, 1988',
  'direcao': 'Susanna Nicchiarelli.',
  'elenco': 'Tryne Dyrholm, John Gordon Sinclair, Anamaria Marinca, Sandor Funtek.',
  'genero': 'musical. Itália, 2017.',
  'duracao': '93 minutos.'},
 {'nome': 'HOTEL TRANSILVÂNIA 3:\xa0 FÉRIAS MONSTRUOSAS',
  'direcao': 'Genndy Tartakovsky.',
  'outro': 'Animação.;',
  'genero': 'comédia. Estados Unidos, 2018.',
  'duracao': '97 minutos.'},
 {'nome': 'MAMMA MIA! LÁ VAMOS NÓS DE NOVO',
  'direcao': 'Ol Parker.',
  'elenco': 'Lily James, Amanda Seyfried, Christine Baranski, Pierce Brosnan.',
  'genero': 'musical. Reino Unido, 2018.',
  'duracao': '114 minutos.'},
 {'nome': 'MISSÃO IMPOSSÍVEL − EFEITO FALLOUT',
  'direcao': 'Christopher McQuarrie.',
  'elenco': 'Tom Cruise, Sean Harris, Henry Cavill, Rebecca Ferguson.',


### Omelete

In [6]:
xpath_movie_title = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[1]/div[2]/div[1]'
xpath_movie_subtitle = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[1]/div[2]/div[2]'
xpath_movie_details1 = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[2]/div/div[2]/div[1]/p'
xpath_movie_details2 = '/html/body/main/div[1]/section[1]/div[3]/div/div/article/div[3]/div[1]/div[1]/article/a/div[2]/div/div[2]/div[2]/p'

filmes=[]
for i in range (11,21):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    r=tree.xpath(xpath_movie_title)
    filme['nome'] = get_element_text(r)
    r=tree.xpath(xpath_movie_details1)+tree.xpath(xpath_movie_details2)
    for item in r:
        categoria = remover_acentos(item.text.replace("Ã§Ã£","çã").strip().lower())
        filme[categoria] = item[0].text.strip()
    filmes.append(filme)
filmes

[{'nome': 'Hotel Artemis',
  'ano:': '2018',
  'classificacao:': '16 anos',
  'duracao:': '93 min',
  'direcao:': 'Drew Pearce',
  'roteiro:': 'Drew Pearce',
  'elenco:': 'Zachary Quinto, Jenny Slate, Jeff Goldblum, Sofia Boutella, Sterling K. Brown, Jodie Foster'},
 {'nome': 'Nasce Uma Estrela',
  'ano:': '2018',
  'classificacao:': '-1 anos',
  'direcao:': 'Bradley Cooper',
  'roteiro:': 'Bradley Cooper, Will Fetters, Irene Mecchi, Christopher Wilkinson, Stephen J. Rivele',
  'elenco:': 'Bradley Cooper, Lady Gaga, Sam Elliott'},
 {'nome': 'Red Joan',
  'classificacao:': '-1 anos',
  'direcao:': 'Trevor Nunn',
  'elenco:': 'Tom Hughes, Stephen Campbell Moore, Sophie Cookson, Judi Dench'},
 {'nome': 'Destroyer',
  'ano:': '2018',
  'classificacao:': '-1 anos',
  'direcao:': 'Karyn Kusama',
  'roteiro:': 'Matt Manfredi, Phil Hay',
  'elenco:': 'Toby Kebbell, Tatiana Maslany, Nicole Kidman'},
 {'nome': 'O Predador (2018)',
  'ano:': '2018',
  'classificacao:': '-1 anos',
  'direcao:': 'S

### CineClick

In [7]:
titulo = '//*[@id="movie"]/div[2]/div/div/div[1]/div[2]/div[2]/h1'
ano_duracao='//*[@id="movie"]/div[2]/div/div/div[1]/div[2]/div[2]/p'
genero = '//*[@id="movie"]/div[2]/div/div/div[1]/div[2]/div[2]/div[2]/p[1]'
direcao='//*[@id="movie"]/div/ul'
details='//*[@id="movie"]/div/p'
elenco='//*[@id="movie"]/div/div/p'

filmes = []
for i in range (21,31):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    
    r=tree.xpath(titulo)
    filme["nome"]=r[0].text
    
    r=tree.xpath(ano_duracao)
    ad=get_element_text(r).strip().replace(" ","")
    try:
        filme["ano"]=ad[:ad.index(",")]
        filme["duracao"]=ad[ad.index(",")+1:]
    except ValueError:
        pass
    
    r=tree.xpath(genero)
    gen=get_element_text(r)
    try:
        filme["genero"]=gen[gen.index(":")+1:].strip()
    except ValueError:
        filme["genero"]=gen
        
    r=tree.xpath(direcao)
    filme["direcao"]=get_element_text(r[0]).strip()
    
    r=tree.xpath(details)
    for item in r:
        info = get_element_text(item)
        try:
            categoria=remover_acentos(info[:info.index(":")].lower())
            filme[categoria]=info[info.index(":")+1:]
        except ValueError:
            pass    
            
    r=tree.xpath(elenco)
    filme["elenco"]=get_element_text(r[0])
    filmes.append(filme)
filmes

[{'nome': 'O PREDADOR',
  'ano': '2018',
  'duracao': '101MIN.',
  'genero': 'Ação',
  'direcao': 'Shane Black',
  'roteiro': 'Fred Dekker, Jim Thomas, John Thomas, Shane Black',
  'producao': 'Blondel Aidoo, John Davis, Lawrence Gordon',
  'fotografia': 'Larry Fong',
  'trilha sonora': 'Henry Jackman',
  'estudio': 'Canada Film Capital, Dark Castle Entertainment, Davis Entertainment, TSG Entertainment, Twentieth Century Fox Film Corporation',
  'montador': 'Harry B. Miller III',
  'distribuidora': 'Fox Film',
  'elenco': 'Alfie Allen, Andrew Jenkins, Augusto Aguilera, Boyd Holbrook, Crystal Mudry, Edward James Olmos, Inka Malovic, Jacob Tremblay, Jake Busey, Keegan-Michael Key, Lochlyn Munro, Niall Matter, Olivia Munn, Patrick Sabongui, Paul Lazenby, RJ Fetherstonhaugh, Sterling K. Brown, Thomas Jane, Trevante Rhodes, Yvonne Strahovski'},
 {'nome': 'A FREIRA',
  'ano': '2018',
  'duracao': '96MIN.',
  'genero': 'Terror',
  'direcao': 'Corin Hardy',
  'roteiro': 'Gary Dauberman, James 

### Cinema em Cena

In [8]:
xpath_details = '//*[@class="critica-elenco"]'
nome='/html/body/div/div/div/h1'
ano='/html/body/div/div/div/table/tbody/tr/td/text()'

# tree = etree.parse('./pages/pag31.html', parser)
# tree.xpath(ano)
filmes = []
for i in range(31,41):
    tree = etree.parse(root + str(i) + '.html', parser)
    
    categoria = ""
    detalhes = {}
    filme = {}
    filme["nome"] = tree.xpath(nome)[1].text.strip()
    
    info = tree.xpath(ano)
    if len(info) == 3:
        filme["ano"] = info[0]
        filme["distribuidora"] = info[2]
    elif len(info) == 2:
        filme["ano"] = info[0]
    
    for x in tree.xpath(xpath_details)[0]:
    #     if i.tag == "div" or i.tag == "br":
    #         flag = False
        if x.tag == "h3":
            categoria = remover_acentos(x.text.lower())
            detalhes[categoria] = []
    #         flag = True
        elif x.tag == "a":
            detalhes[categoria].append(x.text)
    for i in detalhes:
        filme[i] = ', '.join(detalhes[i])    
    filmes.append(filme)
    
filmes

[{'nome': 'Ilha dos Cachorros',
  'ano': '19/07/2018',
  'distribuidora': 'Fox',
  'direcao': 'Wes Anderson',
  'elenco': 'Bryan Cranston, Koyu Rankin, Edward Norton, Scarlett Johansson, Bob Balaban, Greta Gerwig, Akira Takayama, Yoko Ono, Akira Ito, Fisher Stevens, Liev Schreiber, Courtney B. Vance, Roman Coppola, Anjelica Huston, Ken Watanabe, F. Murray Abraham, Tilda Swinton, Frances McDormand, Harvey Keitel, Jeff Goldblum, Bill Murray',
  'roteiro': 'Wes Anderson',
  'producao': 'Wes Anderson, Jeremy Dawson, Scott Rudin, Steven Rales',
  'fotografia': 'Tristan Oliver',
  'musica': 'Alexandre Desplat',
  'montagem': 'Andrew Weisblum, Edward Bursch, Ralph Foster',
  'design de producao': 'Adam Stockhausen, Paul Harrod'},
 {'nome': 'Sicário: Dia do Soldado',
  'ano': '28/06/2018',
  'distribuidora': 'Sony',
  'direcao': 'Stefano Sollima',
  'elenco': 'Benicio Del Toro, Josh Brolin, Isabela Moner, Catherine Keener, Jeffrey Donovan, Matthew Modine, Shea Whigham, Manuel Garcia-Rulfo, Eli

### Jornal da Paraíba

In [9]:
xpath_details = '//*[@id="noticias"]/div[1]/article/div[3]/ul/li'
xpath_title = '//*[@id="noticias"]/div[1]/article/header/h3'

filmes = []
for i in range(41,51):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    name = tree.xpath(xpath_title)
    elems = tree.xpath(xpath_details)
    
    filme["nome"]=get_element_text(name[0])
    for item in elems:
        i = get_element_text(item)
        try:
            categoria = remover_acentos(i[:i.index(":")].lower())
            filme[categoria] = i[i.index(":")+1:].strip()
        except ValueError:
            pass
    filmes.append(filme)
filmes

[{'nome': 'O Mistério do Relógio na Parede',
  'genero': 'Fantasia, Mistério, Terror',
  'censura': '10 - Não recomendado para menores de dez anos',
  'duracao': '104 min',
  'diretor': 'Eli Roth',
  'elenco': 'Cate Blanchett, Colleen Camp, Jack Black, Kyle MacLachlan, Renée Elise Goldsberry'},
 {'nome': 'Buscando…',
  'genero': 'Drama, Mistério, Thriller',
  'censura': '14 - Não recomendado para menores de catorze anos',
  'duracao': '102 min',
  'diretor': 'Aneesh Chaganty',
  'elenco': 'Brad Abrell, Debra Messing, Gabriel D. Angell, John Cho, Joseph Lee, Michelle La, Roy Abramsohn, Sara Sohn'},
 {'nome': 'Benzinho',
  'genero': 'Drama',
  'censura': '12 - Não recomendado para menores de doze anos',
  'duracao': '97 min',
  'diretor': 'Gustavo Pizzi',
  'elenco': 'Adriana Esteves, Arthur Teles Pizzi, César Troncoso, Francisco Teles Pizzi, Karine Teles, Konstantinos Sarris, Luan Teles, Mateus Solano, Otávio Müller, Vicente Demori'},
 {'nome': '22 Milhas',
  'genero': 'Ação',
  'censur

### Cinema com Rapadura

In [10]:
xpath_details = '/html/body/div[2]/div[4]/div/div/div[3]/div[3]/div[2]/p'

filmes = []
for i in range(51,61):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    details = tree.xpath(xpath_details)
    
    nome = get_element_text(details[0])
    if "(" in nome:
        filme['nome']=nome[:nome.index("(")].strip()
        filme['ano']=nome[nome.index("(")+1:nome.index(")")]
        
    diretor = get_element_text(details[1])
    if diretor.rfind("-")>-1:
        filme['diretor']=diretor[diretor.rfind("-")+1:].strip()
        
    roteiro = get_element_text(details[3])
    if roteiro.find(":") > -1:
        filme[remover_acentos(roteiro[:roteiro.find(":")].lower())] = roteiro[roteiro.find(":")+1:].strip()
        
    elenco = get_element_text(details[4])
    if elenco.find(":") > -1:
        filme[remover_acentos(elenco[:elenco.find(":")].lower())] = elenco[elenco.find(":")+1:].strip()
    
    filmes.append(filme)
filmes

[{'nome': 'O Predador',
  'ano': '2018',
  'diretor': 'Shane Black',
  'roteiro': 'Fred Dekker, Shane Black',
  'elenco': 'Boyd Holbrook, Trevante Rhodes, Jacob Tremblay, Keegan-Michael Key, Olivia Munn, Thomas Jane, Alfie Allen, Sterling K. Brown, Augusto Aguilera, Jake Busey, Yvonne Strahovski, Edward James Olmos, Niall Matter, Dean Redman, Steve Wilder, Nikolas Dukic, Andrew Jenkins, Crystal Mudry, Paul Lazenby, Lochlyn Munro'},
 {'nome': 'Benzinho',
  'ano': '2018',
  'diretor': 'Gustavo Pizzi',
  'roteiro': 'Gustavo Pizzi, Karine Teles',
  'elenco': 'Karine Teles, Otávio Müller, Adriana Esteves, Konstantinos Sarris, César Troncoso, Mateus Solano, Camilo Pellegrini, Ariclenes Barroso, Pablo Riera, Lucas Gouvêa, Ingrid Guimarães'},
 {'nome': 'A Barraca do Beijo',
  'ano': '2018',
  'diretor': 'Vince Marcello',
  'roteiro': 'Vince Marcello',
  'elenco': 'Joey King, Joel Courtney, Jacob Elordi, Molly Ringwald, Carson White, Frances Sholto-Douglas, Nathan Lynn, Meganne Young, Byron Lan

### Plano Crítico

In [11]:
def text_normalize(text):
    return normalize("NFKD",text)

In [12]:
xpath_details = '(//*[@id="content-area"]/p)[last()]'

filmes = []
for i in range(61,71):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    details = tree.xpath(xpath_details)
    details = get_element_text(details[0]).split("\n")
    filme['nome'] = text_normalize(details[0])
    for item in details[1:]:
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = text_normalize(item[item.find(":")+1:].strip())
    filmes.append(filme)
filmes

[{'nome': 'Funny Girl – Uma Garota Genial (Funny Girl) – EUA, 1968',
  'direcao': 'William Wyler',
  'roteiro': 'Isobel Lennart (baseado em Funny Girl, escrito por Isobel Lennart, Jule Stynne e Bob Merrill)',
  'elenco': 'Barbra Streisand, Omar Sharif, Kay Medford, Anne Francis, Walter Pidgeon, Lee Allen, Mae Questel, Gerald Mohr, Frank Faylen, Alena Johnston, Thordis Brandt, Arthur Tovey',
  'duracao': '149 min.'},
 {'nome': 'Simplesmente Martha (Bella Martha/Alemanha, 2001)',
  'direcao': 'Sandra Nettelbeck',
  'roteiro': 'Sandra Nettelbeck',
  'elenco': 'Antonio Wannek, August Zirner, Diego Ribon, Idil Üner, Katja Studt, Martina Gedeck, Maxime Foerste, Oliver Broumis, Sergio Castellitto, Sibylle Canonica, Ulrich Thomsen',
  'duracao': '103 min.'},
 {'nome': 'O Mistério do Relógio na Parede (The House with a Clock in Its Walls) EUA, 2018',
  'direcao': 'Eli Roth',
  'roteiro': 'Eric Kripke, John Bellairs',
  'elenco': 'Owen Vaccaro, Jack Black, Cate Blanchett, Kyle MacLachlan, Col

### Plano Aberto

In [13]:
xpath_details = '//*[@id="fica-tecnica"]/div[2]/p'

filmes = []
for i in range(71,81):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    details = tree.xpath(xpath_details)
    
    filme['nome'] = get_element_text(details[0])
    for item in details[1:]:
        item = get_element_text(item)
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = item[item.find(":")+1:].strip()
    filmes.append(filme)
filmes

[{'nome': 'Bloqueio, 2018 - Brasil',
  'direcao': 'Victória Álvares, Quentin Delaroche',
  'elenco': 'Marocs da Silva dos Santos, Benedito Amauro, Parazinho'},
 {'nome': 'New Life S.A., 2018 – Brasil',
  'direcao': 'André Carvalheira',
  'roteiro': 'André Carvalheira, Aurélio Aragão, Pablo Gonçalo',
  'elenco': 'Fernanda Rocha, Murilo Grossi, Catarina Accioly, Eric Naves, Edu Moares, Juliano Coacci, André Deca, Bianca Terraza, Wellington Abreu, Marcelo Pelucio',
  'fotografia': 'Krishna Schmidt',
  'design de producao': 'Maíra Carvalho'},
 {'nome': 'Excelentíssimos, 2018 - Brasil', 'direcao': 'Douglas Duarte'},
 {'nome': 'Torre das Donzelas, 2018 - Brasil',
  'direcao': 'Susanna Lira',
  'roteiro': 'Susanna Lira'},
 {'nome': 'Domingo, 2018 – Brasil',
  'direcao': 'Fellipe Barbosa, Clara Linhart',
  'roteiro': 'Lucas Paraizo',
  'elenco': 'Itala Nardi, Camila Morgado, Augusto Madeira, Michael Wahrmann, Martha Nowill, Clemente Viscaíno, Ismael Caneppele'},
 {'nome': 'Carnivores, 2018 - F

### Cinema Sim

In [14]:
xpath_details = '//*[@id="site-top"]/div[1]/div[4]/div/div/main/article/div[1]/div[1]/div[1]/div[2]'
portuguese_name = '//*[@id="site-top"]/div[1]/div[3]/div[2]/div/h1'

filmes = []
for i in range(81,91):
    filme = {}
    tree = etree.parse(root + str(i) + '.html', parser)
    
    nome = tree.xpath(portuguese_name)[0].text
    if nome.find("|") > -1:
        filme['nome'] = nome[:nome.find("|")].strip()
    else:
        filme['nome'] = nome
    
    details = tree.xpath(xpath_details)
    details = details[0].text.split('\n')
    for item in details:
        if item.find(":") > -1:
            filme[remover_acentos(item[:item.find(":")].lower())] = item[item.find(":")+1:].strip()
        
    filmes.append(filme)
filmes

[{'nome': 'O Retorno do Herói',
  'titulo original': 'Le Retour du Héros',
  'lancamento': '20 de setembro',
  'direcao': 'Laurent Tirard',
  'roteiro': 'Laurent Tirard, Grégoire Vigneron',
  'genero': 'Comédia',
  'elenco': 'Jean Dujardin, Mélanie Laurent, Noémie Merlant',
  'nacionalidade': 'França, Bélgica'},
 {'nome': 'Marvin',
  'titulo original': 'Marvin ou La Belle Éducation',
  'lancamento': '06 de setembro',
  'direcao': 'Anne Fontaine',
  'roteiro': 'Anne Fontaine, Pierre Trividic',
  'genero': 'Drama',
  'elenco': 'Finnegan Oldfield, Grégory Gadebois, Vincent Macaigne',
  'nacionalidade': 'França'},
 {'nome': 'Sierra Burgess é uma Loser',
  'titulo original': 'Sierra Burgess is Loser',
  'lancamento': '07 de setembro de 2018',
  'direcao': 'Ian Samuels',
  'roteiro': 'Lindsey Beer',
  'genero': 'Comédia Romântica',
  'elenco': 'Shannon Purser, RJ Cyler, Noah Centineo, Will Peltz, Kristine Froseth, Lea Thompson e Alan Ruck',
  'nacionalidade': 'Estados Unidos'},
 {'nome': 'Cr

### Pocilga

In [22]:
xpath_details = '//*[@id="post-12790"]/div[2]/blockquote[2]/p'
xpath_title = '//*[@id="post-12777"]/div[2]/h4/strong'
'//*[@id="post-12790"]/div[2]/h4/strong'

# filmes = []
# for i in range(91,92):
#     filme = {}
#     tree = etree.parse(root + str(i) + '.html', parser)
#     details = tree.xpath(xpath_details)

tree = etree.parse('./pages/pag91.html', parser)
details = tree.xpath(xpath_title)

In [20]:
# get_element_text(details[0]).split('\n')
# get_element_text(details[0])

IndexError: list index out of range

In [23]:
>>> regexpNS = "http://exslt.org/regular-expressions"
>>> find = etree.XPath("//*[re:test(., '^abc$', 'i')]",
...                    namespaces={'re':regexpNS})

>>> root = etree.XML("<root><a>aB</a><b>aBc</b></root>")
>>> print(find(root)[0].text)

0
